<div style="font-weight: bold; color:#5D8AA8" align="center">
    <div style="font-size: xx-large">Machine Learning with Funcitonal Data</div><br>
    <div style="font-size: x-large; color:gray">Introduction to FDA with scikit-fda</div><br>
    <div style="font-size: large">José Luis Torrecilla Noguerales - Universidad Autónoma de Madrid</div><br></div><hr>
</div>

**Initial setting**: This cell defines the Notebook setting

In [ ]:
%%html
<style>
    .qst {background-color: #b1cee3; padding:10px; border-radius: 5px; border: solid 2px #5D8AA8;}
    .qst:before {font-weight: bold; content: "Questions"; display: block; margin: 0px 10px 10px 10px;}
    h1, h2, h3 {color: #5D8AA8;}
    .text_cell_render p {text-align: justify; text-justify: inter-word;}
</style>

**Packages to use:**

In [ ]:
import skfda
import matplotlib.pyplot as plt
import sklearn

# The class FDataGrid
## Defining the [FDataGrid](https://fda.readthedocs.io/en/latest/modules/autosummary/skfda.representation.grid.FDataGrid.html#skfda.representation.grid.FDataGrid)

The main attributes when defining dataset of discretized functional data are **grid_points** and **data_matrix** which stand for the array of discretization points $t_1,\ldots,t_M$, and the matrix with the values of the $N$ trajectories evaluated at the grid points (one observation per row).  the values of the trajectories at the grid points. 

Note that the grid points can be omitted, and in that case, they are automatically assigned as equispaced points in domain set.


In [ ]:
#grid_points = [0.0, 0.2, 0.4, 0.5, 0.6, 1.0]  # Grid points of the curves
data_matrix = [
    [0.00, 0.20, 0.50, 0.90, 1.20, 1.00],  # First observation
    [0.00, 0.04, 0.25, 0.75, 0.85, 1.00],  # Second observation
    [0.25, 0.00, 1.00, 0.00, 0.50, 1.00],  # Third observation
]

fd = skfda.FDataGrid(
#    grid_points=grid_points,
    data_matrix=data_matrix
)

fd.scatter(s=0.3) #Discretized values
plt.show()

fig = fd.plot() #Interpolated functions. Depends on the interpolation attribute
fd.scatter(fig=fig)
plt.show()
fd

## Examples in higher dimensions

We will load the digits dataset from *scikit-learn*, which contains images of the digits from $0$ to $9$. This can be loaded with [load_digits](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits), returning a NumPy array. We can view this digits as surfaces, that is, functions $x(s,t):\mathbb{R}^2 \longrightarrow\mathbb{R}$. The problem is that the data has been flattened into a 1D vector of pixels, so first, we need to reshape them to their original 8x8 shape. 

In [ ]:
#Directive to get interactive plots
#Package ipympl is needed. You need to install the package once with !pip install ipympl
%matplotlib widget 

In [ ]:
from sklearn.datasets import load_digits

X, y = load_digits(return_X_y=True)
print(X.shape)
X = X.reshape(-1, 8, 8)
print(X.shape)

fd2 = skfda.FDataGrid(X)

# Plot the first 2 observations
fd2[0].plot()
plt.show()
fd2[7].plot()
plt.show()

## Interpolation and Extrapolation

For the evaluation between the points of discretization, or sample points, is necessary to interpolate. By default it is used linear interpolation, which is one of the simplest methods of interpolation and therefore one of the least computationally expensive, but has the disadvantage that the interpolant is not differentiable at the points of discretization.

The interpolation method of the *FDataGrid* could be changed setting the **attribute interpolation**. Once we have set an interpolation it is used for the evaluation of the object.

In [ ]:
from skfda.representation.interpolation import SplineInterpolation


fig, axes = plt.subplots(1, 3, figsize=(10, 2.5))


fd.scatter(axes[0],s=0.3) #Discretized values

fd.plot(axes[1]) #Interpolated functions. Depends on the interpolation attribute (linear by default)
fd.scatter(axes[1])

#Polynomial interpolation
fd.interpolation = SplineInterpolation(interpolation_order=3)

fd.plot(axes[2])
fd.scatter(axes[2])
fig.suptitle(None)
plt.show()

<div class="qst">

* How does interpolation affect surfaces? Let's consider the example of digits.
    
* What happens when we change the interpolation order of B-splines to 0?

</div>


Extrapolation refers to the process of estimating values beyond the range of available data.  The extrapolation criterion can be specified when the object is created or changing the **attribute extrapolation**.

If the extrapolation is not specified when a list of points is evaluated and the default extrapolation of the objects has not been specified it is used the type “none”, which will evaluate the points outside the domain without any kind of control.

For this reason the behavior outside the domain will change depending on the representation, obtaining a periodic behavior in the case of the Fourier basis and polynomial behaviors in the rest of the cases.

In [ ]:
domain_extended = (-0.2, 1.2)

fd.extrapolation = "None" #Default extrapolation strategy
fig = fd.plot(domain_range=domain_extended, linestyle="--")
plt.gca().set_prop_cycle(None) 
fd.plot(fig=fig)
plt.show()

fd.extrapolation = "bounds" 
fig = fd.plot(domain_range=domain_extended, linestyle="--")
plt.gca().set_prop_cycle(None) 
fd.plot(fig=fig)
plt.show()

fd.extrapolation = "periodic"
fig = fd.plot(domain_range=domain_extended, linestyle="--")
plt.gca().set_prop_cycle(None) 
fd.plot(fig=fig)
plt.show()

## Loading functional datasets: the fetch functions
### Common datasets

Some commonly used datasets have their own fetch functions. Here we explore **aemet**, wich consists of daily summaries from 73 spanish weather stations during the period 1980-2009. The dataset contains the geographic information of each station and the average for the period 1980-2009 of daily temperature, (log)precipitation and wind speed. This can be seen as an example o the so-called multivariate functional data.

In [ ]:
X, _ = skfda.datasets.fetch_aemet(return_X_y=True)

print(repr(X)) #repr returns a more detailed descrition of the object

X.plot()
plt.show()

In [ ]:
#Adding some interactivity
from matplotlib.widgets import Slider

graph_plot = skfda.exploratory.visualization.representation.GraphPlot(X)
interactive_plot = skfda.exploratory.visualization.MultipleDisplay(
    [graph_plot],           #plots included
    criteria=range(len(X)), #
    sliders=Slider,
    label_sliders=["ID"]
)
interactive_plot.plot()
plt.show()

In [ ]:
X.coordinates[0].plot() #Selecting temperatures only
plt.show()

### Datasets from repositories

There are functional datasets in many repositories. Functions **fetch_cran** and **fetch_ucr** make it easy for us to access data in in the CRAN repository (R), and in the UEA & UCR Time Series Classification Repository repositories, respectively. Note these datasets do not follow a particular structure, so you will need to know how it is structured internally in order to use it properly.

In [ ]:
X, y = skfda.datasets.fetch_ucr("GunPoint", return_X_y=True)
X.plot(group=y)
plt.show()

## Generation of random trajectories
ddd

In [ ]:
from skfda.misc.covariances import Brownian

fd = skfda.datasets.make_gaussian_process(
start = 0,                 #initial point
stop = 1,                   #final point
n_samples=10,               #sample size
n_features=100,             #number of variables/discretization points
mean=0,        #mean function. lambda function f(t) = t^2
cov=Brownian(variance=1),   #covariance function
random_state=0              #seed
)

fd.plot() 
plt.show()

<div class="qst">

* Try to draw 10 trajectories of a standard Brownian motion.
    
    
* Do you know how to compute $5^2$ using lambda functions?
</div>

In [ ]:
Brownian() #Interaction with jupyter (prototype)

# The class FDataBasis
### From FDataGrid to FDataBasis
The method **to_basis()** transform FDataGrid objects to a basis representation (FDataBasis). The desired basis must be passed as an argument.  you will need to call the method to_basis, passing the desired basis as an argument. The discretized trajectories will be projected to the functional basis by solving a least squares problem in order to find the optimal coefficients of the expansion. 
The reciprocal action can be dan with **to_grid**. This method evaluates the functions in a grid supplied as an argument.

In [ ]:
X, y = skfda.datasets.fetch_phoneme(return_X_y=True)

X = X[:5] # Select only the first 5 observations

X.plot()
plt.show()

#B-Spline basis. The parameter order refers to the degree of the polynomials plus 1.
basis = skfda.representation.basis.BSplineBasis(n_basis=8, order=4)
X.to_basis(basis).plot()
plt.show()


<div class="qst">

* Explore the effect of varying the number of elements in the basis and the type of basis.
* What happens if we extrapolate the basis set to the interval (-1,10)? How does extrapolation behave when we change the basis?

</div>

# Smoothing
### Kernel smoothing
Although the basis representation results in the smoothing of the trajectories, there are other smoothing techniques that do not require the use bases. Perhaps the most popular are those based on kernels like Nadaraya-Watson or near neighbors. Some of these methods are available in the scikit-fda's class **KernelSmoother**.

In [ ]:
from skfda.misc.hat_matrix import (
    KNeighborsHatMatrix,
    NadarayaWatsonHatMatrix
)

from skfda.preprocessing.smoothing import KernelSmoother

X_s = KernelSmoother(
    kernel_estimator=NadarayaWatsonHatMatrix(bandwidth=0.5),
).fit_transform(X)


X_s = KernelSmoother(
    kernel_estimator=KNeighborsHatMatrix(n_neighbors=50),
).fit_transform(X)

X_s.plot()
plt.show()

<div class="qst">
    
* What happens when varying the bandwith parameter?
    
* Try Nearest neighbors kernel. What happens with large values of $k$?
</div>

### Bandwith selection
Let's build a synthetic example to see clearly the effects of overfitting and oversmoothing. The example will be a sinusoidal trajectory with some noise and our objective will be to recover the original signal.

In [ ]:
from skfda.datasets import make_sinusoidal_process

#Original trajectory, without noise
x_original = make_sinusoidal_process(
    n_samples=1,
    n_features=20,
    error_std=0,
    random_state=1)
fig = x_original.plot()

#Noisy trajectory
x_noisy = make_sinusoidal_process(
    n_samples=1,
    n_features=20,
    error_std=0.2,
    random_state=1)
x_noisy.scatter(fig=fig)
x_noisy.plot(fig=fig)
plt.show()

In [ ]:
fig = x_original.plot()
x_noisy.scatter(fig=fig)

#Underfitting - oversmoothing
x_s1 = KernelSmoother(
    kernel_estimator=NadarayaWatsonHatMatrix(bandwidth=0.3),
).fit_transform(x_noisy)
x_s1.plot(fig=fig)

#Overfitting - undersmoothing
x_s2 = KernelSmoother(
    kernel_estimator=NadarayaWatsonHatMatrix(bandwidth=0.05),
).fit_transform(x_noisy)
x_s2.plot(fig=fig)

fig.legend(
    [
        'original trajectory',
        'noisy observations',
        'Underfitting',
        'Overfitting',
    ]    
)
plt.show()

### Generalized cross-validation
In general, we do not know the best value for the smoothing parameter, and trial and error is not the best solution. There are cross-validation techniques to automate the choice of the bandwith by minimizing certain objective functions. In the case of scikit-fda we can use a [generalized cross validation](https://fda.readthedocs.io/en/latest/modules/preprocessing/autosummary/skfda.preprocessing.smoothing.validation.LinearSmootherGeneralizedCVScorer.html#skfda.preprocessing.smoothing.validation.LinearSmootherGeneralizedCVScorer).

In [ ]:
from skfda.preprocessing.smoothing.validation import SmoothingParameterSearch
from skfda.preprocessing.smoothing.validation import akaike_information_criterion, LinearSmootherGeneralizedCVScorer
import numpy as np

# Defining grid search
bandwidth = np.linspace(0.01, 1, 100)


# Nadaraya-Watson kernel smoother
nw = SmoothingParameterSearch(
    KernelSmoother(kernel_estimator=NadarayaWatsonHatMatrix()),
    bandwidth,
    scoring=LinearSmootherGeneralizedCVScorer(penalization_function=lambda t:1),
    param_name='kernel_estimator__bandwidth'
)

# Fit and transform must be done separately since SmoothingParameterSearch
# object do not have fit_transform method
nw.fit(x_noisy)
x_opt = nw.transform(x_noisy)


fig = x_original.plot()
x_noisy.scatter(fig=fig)
x_opt.plot(fig=fig)

plt.show()

In [ ]:
np.linspace(0.01, 1, 100)

<div class="qst">
    
* Is it possible to obtain a better fit?
</div>

# Derivatives
The computation of derivatives is of particular importance in functional data analysis. In the scikit-fda package, the method derivative() can be used to per-
form this operation for both FDataGrid and FDataBasis objects. In the case of FDataGrid
objects, derivatives are approximated using finite differences. For FDataBasis objects, they
are computed exactly in terms of the derivatives of the basis functions. Therefore, if a new
type of basis is designed, it is necessary to implement the derivatives of the basis functions in
the corresponding class.

Here, we illustrate the use of the derivatives with the popular Tecator dataset.

In [ ]:
X, y = skfda.datasets.fetch_tecator(return_X_y=True)
y = y[:, 0] # keeping only the fat content as target variable

X.plot(gradient_criteria=y)
plt.show()

X_der = X.derivative(order=2) #Second derivative by finite differences
X_der.plot(gradient_criteria=y)
plt.show()